In [1]:
!pip install mmsegmentation
!pip install mmcv-full

In [2]:

import cv2
import numpy as np
import random
from pathlib import Path
import matplotlib.pyplot as plt
from mmseg.apis import init_segmentor, inference_segmentor
from sklearn.manifold import TSNE
import torchvision
import mmcv

c:\Users\mokom\Anaconda3\envs\data_science\lib\site-packages\mmcv\__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [3]:

config_file = 'https://github.com/open-mmlab/mmsegmentation/blob/master/configs/deeplabv3plus/deeplabv3plus_r50-d8_512x1024_40k_cityscapes.py'
checkpoint_file = 'https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3plus/deeplabv3plus_r50-d8_512x1024_40k_cityscapes/deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth'

In [4]:
!pip install wget
# google colaboで実行する場合は”wget $config_file”でOK
!python -m wget $config_file
!python -m wget $checkpoint_file


Saved under deeplabv3plus_r50-d8_512x1024_40k_cityscapes (1).py

Saved under deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd (1).pth


In [5]:
mmseg_path = Path('../../mmsegmentation')
config_file_path = mmseg_path / 'configs/deeplabv3plus/deeplabv3plus_r50-d8_512x1024_40k_cityscapes.py'
checkpoint_file_path = './deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth'
cfg = mmcv.Config.fromfile(config_file_path)

# avoid TypeError: init_segmentor() got an unexpected keyword argument 'norm_cfg'
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
model = init_segmentor(cfg, checkpoint_file_path, device='cpu')

img = 'test.JPEG'
result = inference_segmentor(model, img)
# model.show_result(img, result, show=True)

c:\Users\mokom\Anaconda3\envs\data_science\lib\site-packages\mmseg\models\losses\cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '


load checkpoint from local path: ./deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth


c:\Users\mokom\Anaconda3\envs\data_science\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [6]:
def middle_layer_output(target, inputs):
    feature = None

    def forward_hook(module, inputs, outputs):
        global features
        features = outputs.detach().clone()

    handle = target.register_forward_hook(forward_hook)
    model.eval()
    inference_segmentor(model, inputs)
    handle.remove()
    return features

In [7]:
target = model.backbone.layer4[-1].conv3
middle_layer_output(target, img)

tensor([[[[ 0.0413,  0.0282,  0.0322,  ...,  0.0251,  0.0164,  0.0279],
          [ 0.0436,  0.0229,  0.0365,  ...,  0.0221,  0.0101,  0.0261],
          [ 0.0401,  0.0246,  0.0351,  ...,  0.0294,  0.0168,  0.0293],
          ...,
          [ 0.0241,  0.0201,  0.0254,  ...,  0.0296,  0.0280,  0.0334],
          [ 0.0261,  0.0204,  0.0220,  ...,  0.0323,  0.0271,  0.0309],
          [ 0.0222,  0.0175,  0.0217,  ...,  0.0259,  0.0178,  0.0254]],

         [[ 0.0320,  0.0264,  0.0326,  ...,  0.0314,  0.0302,  0.0330],
          [ 0.0340,  0.0345,  0.0351,  ...,  0.0264,  0.0272,  0.0288],
          [ 0.0342,  0.0298,  0.0356,  ...,  0.0284,  0.0292,  0.0278],
          ...,
          [ 0.0357,  0.0351,  0.0384,  ...,  0.0469,  0.0461,  0.0438],
          [ 0.0382,  0.0344,  0.0373,  ...,  0.0335,  0.0359,  0.0351],
          [ 0.0383,  0.0380,  0.0399,  ...,  0.0337,  0.0380,  0.0346]],

         [[ 0.0437,  0.0390,  0.0484,  ...,  0.0461,  0.0375,  0.0451],
          [ 0.0562,  0.0455,  